In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import losses
import numpy as np
import torch
import pydot

from numpy.random import seed
seed(42)

#OR = Original reviews (presumably human created and authentic); CG = Computer-generated fake reviews.
df = pd.read_csv(
    r"C:\Users\Mark\Desktop\reviewDetector-main\python\fakereviews.csv",
    names=["category","rating","label","text_"])


#Computer generierte Texte bekommen die Zahl 1 für spam zugeordnet. Im Grunde wird das Label in diesem und
#im nächsten Schritt nur als Zahl dargestellt zur einfacheren Verarbeitung.
df['labelzahl'] = df['label'].apply(lambda x: 1 if x=='CG' else 0)

df['rating'] = df['rating'].apply(lambda x: 1 if x=='1.0' else 2 if x=='2.0' else 3 if x=='3.0' else 4 if x=='4.0'
                                       else 5) 
"""if x=='5.0'"""


datenset_xy = df[["rating","text_","labelzahl"]]

print(datenset_xy)

print(datenset_xy.size)


def get_dataset_partitions_tf(ds=datenset_xy, train_split=0.8, val_split=0.1, test_split=0.1):
    assert (train_split + test_split + val_split) == 1

    train_ds=ds.sample(frac=0.8)
    val_test_ds=ds.drop(train_ds.index)
    val_ds=val_test_ds.sample(frac=0.5)
    test_ds=val_test_ds.drop(val_ds.index)

    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(ds=datenset_xy, train_split=0.8, val_split=0.1, test_split=0.1)


print("Größe der Datensets")
print(train_ds.size)
print(val_ds.size)
print(test_ds.size)


train_text = np.asarray(train_ds.pop('text_'))
train_rating = np.asarray(train_ds.pop('rating'))
train_label = np.asarray(train_ds.pop('labelzahl'))

print(train_text)
print(train_rating)
print(train_label)

val_text = np.asarray(val_ds.pop('text_'))
val_rating = np.asarray(val_ds.pop('rating'))
val_label = np.asarray(val_ds.pop('labelzahl'))

test_text = np.asarray(test_ds.pop('text_'))
test_rating = np.asarray(test_ds.pop('rating'))
test_label = np.asarray(test_ds.pop('labelzahl'))


max_features = 10000
sequence_length = 250

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)


# The vocabulary for the layer must be either supplied on construction or learned via adapt().
vectorization_layer = vectorize_layer.adapt(train_text)
vectorization_layer = vectorize_layer.adapt(val_text)
vectorization_layer = vectorize_layer.adapt(test_text)

train_text = np.array(vectorize_layer(train_text))
val_text = np.array(vectorize_layer(val_text))
test_text = np.array(vectorize_layer(test_text))

#################

train_text = tf.convert_to_tensor(train_text)
train_rating = tf.convert_to_tensor(train_rating)
train_label = tf.convert_to_tensor(train_label)

train_text = tf.cast(train_text, tf.int64)
train_rating = tf.cast(train_rating, tf.int64)
train_label = tf.cast(train_label, tf.int64)

################

val_text = tf.convert_to_tensor(val_text)
val__rating = tf.convert_to_tensor(val_rating)
val_label = tf.convert_to_tensor(val_label)

val_text = tf.cast(val_text, tf.int64)
val__rating = tf.cast(val_rating, tf.int64)
val_label = tf.cast(val_label, tf.int64)


embedding_dim = 16


train_text = np.asarray(train_text).astype(np.int64)

train_rating = np.asarray(train_rating).astype(np.int64)
train_rating = np.expand_dims(train_rating, axis=1)
train_rating = np.pad(train_rating, ((0,0),(0,249)), mode='edge')

train_label = np.asarray(train_label).astype(np.int64)



val_text = np.asarray(val_text).astype(np.int64)

val_rating = np.asarray(val_rating).astype(np.int64)
val_rating = np.expand_dims(val_rating, axis=1)
val_rating = np.pad(val_rating, ((0,0),(0,249)), mode='constant', constant_values=1)

val_label = np.asarray(val_label).astype(np.int64)


print(np.array(train_text).shape)
print(np.array(train_rating).shape)
print(np.array(train_label).shape)

text_input = tf.keras.Input(
    shape=(250,), name = "text"
)


rating_input = tf.keras.Input(
    shape=(250,), name = "rating"
)

textembedding = layers.Embedding(max_features + 1, embedding_dim)(text_input)
textembedding = layers.Dropout(0.2)(textembedding)
textembedding = layers.GlobalAveragePooling1D()(textembedding)
textembedding = layers.Dropout(0.2)(textembedding)


rating = layers.Dropout(0.2)(rating_input)

concat = layers.concatenate([textembedding,  rating])

auswertung = layers.Dense(1,name="label")(concat)

model = tf.keras.Model(
    inputs=[text_input, rating_input],
    outputs=[auswertung],
)

model.summary()

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=losses.BinaryCrossentropy(from_logits=True),
    loss_weights=[1.0],
    metrics=tf.metrics.BinaryAccuracy(threshold=0.0),
)


history = model.fit(
    {"text": train_text, "rating": train_rating},
    {"label": train_label},
    epochs=200,
    batch_size=32,
)

loss, accuracy = model.evaluate(
    {"text": val_text, "rating": val_rating},
    {"label": val_label})

print("Loss und Accuracy des Validationssets")
print("Loss: ", loss)
print("Accuracy: ", accuracy)


##### Modell laden und predictions in anderer Datei

model.save('amazon_zwei_inputs.keras')



print("Prediction Dataset ausgeben")
predict_ds = datenset_xy.iloc[25:85]

print(predict_ds)


predict_text = np.asarray(predict_ds.pop('text_'))
print("predict text ausgeben")
print(predict_text)
predict_text = tf.convert_to_tensor(predict_text)
predict_text = np.array(vectorize_layer(predict_text))
predict_text = tf.cast(predict_text, tf.int64)
predict_text = np.asarray(predict_text).astype(np.int64)

predict_rating = np.asarray(predict_ds.pop('rating'))
predict_rating = tf.convert_to_tensor(predict_rating)
predict_rating = tf.cast(predict_rating, tf.int64)
predict_rating = np.asarray(predict_rating).astype(np.int64)
predict_rating = np.expand_dims(predict_rating, axis=1)
predict_rating = np.pad(predict_rating, ((0,0),(0,249)), mode='edge')



prediction = model.predict(
   x={"text": predict_text, "rating": predict_rating})


print(prediction)



       rating                                              text_  labelzahl
0           5                                              text_          0
1           5  Love this!  Well made, sturdy, and very comfor...          1
2           5  love it, a great upgrade from the original.  I...          1
3           5  This pillow saved my back. I love the look and...          1
4           1  Missing information on how to use it, but it i...          1
...       ...                                                ...        ...
40428       4  I had read some reviews saying that this bra r...          0
40429       5  I wasn't sure exactly what it would be. It is ...          1
40430       2  You can wear the hood by itself, wear it with ...          0
40431       1  I liked nothing about this dress. The only rea...          1
40432       5  I work in the wedding industry and have to wor...          0

[40433 rows x 3 columns]
121299
Größe der Datensets
97038
12132
12129
["Eva is on her t